# Streamlit App을 EC2에서 서비스 

1. App을 Github에 올린다.
    - Github에 올릴 내용
    	- App source
    	- requirements.txt: 설치할 lib 목록
	- openai api 키와 같은 비용이 들어가는 설정 정보는 github에 올리지 않는다.
2. [EC2 instance에 인바운드(inbound) 규칙 추가](02_EC2생성.ipynb#보안그룹의-인바운드(Inbound)-규칙-추가하기)
    - streamlit을 서비스할 port를 열어준다.(EC2 instance 생성 할때 설정하지 않은 경우)
        - streamlit 기본 포트: 8501
    - streamlit 실행시 포트 변경
        - `streamlit run app.py --server.port <포트번호>`
        - 만약 포트번호를 지정해서 실행할 경우 그 포트번호로 **인바운드 규칙**을 설정한다.
3. Streamlit 실행
    - `nohup streamlait run 실행파일.py & `

# Django Project를 EC2에서 서비스

## nginx 설치
- Nginx는 오픈소스 웹서버 프로그램이다.
- Django 웹 애플리케이션의 성능, 안정성, 보안을 위해 nginx와 연동해 서비스를 한다.
- Nginx는 http 요청을 받는 것, http 응답을 생성하는 역할과 static 파일 서비스를 담당한다.
- Django Application(View)은 WSGI와 통신해서 실행한다.
  
```bash
sudo apt update
sudo apt upgrade
sudo apt install python3-pip python3-venv nginx
```

## 가상 환경 생성 및 프로젝트 세팅
- Django 설치
- gunicorn 설치
    - WSGI(Web Server Gateway Interface) 서버
    - 웹서버의 HTTP 요청을 파이썬 애플리케이션에 전달하고, 애플리케이션이 생성한 응답을 다시 웹 서버로 반환하는 역할을 한다.
- **WSGI의 작동 방식**
    - 클라이언트가 HTTP 요청을 보낸다.
    - 웹 서버(Nginx, Apache 등)가 요청을 받아 WSGI 서버(Gunicorn, uWSGI 등)에 전달한다.
    - WSGI 서버는 요청을 파이썬 애플리케이션(Django, Flask 등)에 전달한다.
    - 애플리케이션은 요청을 처리하고 응답을 생성하여 WSGI 서버로 반환한다.
    - WSGI 서버는 응답을 웹 서버로 전달하고, 웹 서버는 Http응답정보를 클라이언트에 반환됩니다

![img](figures/python/nginx_gunicorn.png)

- **구문**
```bash
conda create -n django python=3.12
conda activate django
pip install django gunicorn
```

## Django 프로젝트 설정 (중요)
`/home/ubuntu/mypoll/config/settings.py` 수정:
- ALLOWED_HOSTS 설정
    - 서비스할 host를 설정
    - EC2 instance의 public ip 주소.
    - '*'
- `DEBUG=False` 설정
   - 운영시 디버그 모드를 False로 변경한다.
```python
DEBUG = False
ALLOWED_HOSTS = ['*']
```

- STATIC 및 MEDIA 파일 설정:
```python
STATIC_ROOT = BASE_DIR / 'static_collection'
MEDIA_ROOT = BASE_DIR / 'media'
STATIC_URL = '/static/'
MEDIA_URL = '/media/'
```

- Collect static 실행:
```bash
python manage.py collectstatic
```

## Gunicorn 실행 (테스트)
```bash
cd /home/ubuntu/mypoll

gunicorn --bind 0.0.0.0:8000 config.wsgi:application
```

- 웹 브라우저에서 `http://your-ec2-public-ip:8000`로 접속하여 확인.

## Gunicorn 서비스 생성 (컴퓨터 시작시 자동 실행 하도록 설정)
- 서비스 설정파일 생성

```bash
sudo nano /etc/systemd/system/gunicorn.service
```
- 다음 항목 입력
  
```ini
[Unit]
Description=Gunicorn daemon for Django Project
After=network.target                 # 네트워크가 활성화된 후 시작

[Service]
User=ubuntu                          # 서비스 실행 사용자
Group=ubuntu                         # 서비스 실행 사용자 그룹
WorkingDirectory=/home/ubuntu/mypoll # 작업 디렉토리
ExecStart=/home/ubuntu/miniconda3/envs/django/bin/gunicorn --workers 3 --bind unix:/home/ubuntu/mypoll/gunicorn.sock config.wsgi:application  # gunicorn 실행명령어

[Install]
WantedBy=multi-user.target      # 다중 사용자 모드에서 시작
```

- 서비스 시작
```bash
sudo systemctl start gunicorn      # 서비스로 시작
sudo systemctl enable gunicorn     # 자동등록
sudo systemctl status gunicorn     # 상태확인
```

## Nginx 설정
- site 설정파일 생성

```bash
sudo nano /etc/nginx/sites-available/mypoll
```

- 다음 설정을 추가:
    - 주의: `location /static/`, `location /media/` 경로 설정시 뒤에 `/` 생략 하면 안됨
```nginx
server {
    listen 80;
    server_name <<ec2-public-ip>>;

    location /static/ {
        alias /home/ubuntu/mypoll/static_collection/;     
    }

    location /media/ {
        alias /home/ubuntu/mypoll/media/;                
    }

    location / {
        include proxy_params;
        proxy_pass http://unix:/home/ubuntu/mypoll/gunicorn.sock;
    }
}
```


심볼릭 링크 생성 및 Nginx 재시작:
```bash
sudo ln -s /etc/nginx/sites-available/mypoll /etc/nginx/sites-enabled
sudo nginx -t
sudo systemctl restart nginx
```

## 권한 설정 변경
- nginx 에서 `gunicorn.sock` 을 읽을 수있도록 다음 경로의 권한 설정 변경
```bash
sudo chmod 755 /home/ubuntu     # 소유자/그룹/외부 모두 읽기 가능하도록 설정
```

## 코드/설정 변경시
- django 코드 변경시 `gunicorn`, `nginx` 재시작 한다.
```bash
sudo systemctl daemon-reload          # 설정파일 변경시 실행.
sudo systemctl restart gunicorn
sudo systemctl restart nginx
```

## 서버 상태 확인 및 로그 모니터링
- **서비스 상태 확인**
```bash
sudo systemctl status gunicorn
sudo systemctl status nginx
```
- **로그 확인**
```bash
tail /var/log/nginx/error.log
```

# Django Application - RDS DB 연결

1. [RDS에 MySQL DB 설치](04_RDS.ipynb#데이터베이스-생성-설정)
2. `pymysql` 설치
```bash
conda activate django
pip install pymysql
```

3. DB 연결 후 database 생성
   - `mysql -u 계정 -p -h 앤드 포인트`
   - 연결 후
       - Database 확인: `show databases`
       - Database 생성: `create database mypoll`

4. **settings.py** 설정
    
    - **mysql lib 설정**
        ```python
        import pymysql
        pymysql.install_as_MySQLdb()
        ```
    
    - **Database 연결 설정***
        ```python
        DATABASES = {
            'sqlite3': {
                'ENGINE': 'django.db.backends.sqlite3',
                'NAME': BASE_DIR / 'db.sqlite3',
            },
            'default': {
                'ENGINE': 'django.db.backends.mysql',
                'NAME': '<<Database 이름>>',
                'USER': '<<username>>',
                'PASSWORD': '<<password>>',
                'HOST': '<<RDS DB instance end point>>',
                'PORT': '3306',
            }
        }
        ```

5. 서비스 reload
```bash
sudo systemctl daemon-reload     
sudo systemctl restart gunicorn
sudo systemctl restart nginx
```